# Model Development - Solar CAPEX Estimator

This notebook develops a machine learning model to predict total installed costs (CAPEX) for commercial solar installations using the LBNL Tracking the Sun dataset. It has five main sections to guide the process from data loading to model evaluation, which will eventually be used in a SolarCapexEstimator class. The five sections are:

1) **DataLoader**, responsible for loading Tracking the Sun data from CSV files and filtering it to rows relevant to our use case (commercial solar installations in the US).

2) **DataCleaner**, responsible for cleaning the data by removing rows with missing or invalid values in the target column (total installed price). It also drops columns where most values are missing or fills in missing values with appropriate strategies.

3) **FeatureEngineer**, responsible for creating new features from the existing data that may help the model learn better, but are still readable and interpretable by users.

4) **ModelTrainer**, responsible for training a machine learning model (e.g. linear regression, random forest, or gradient boosting) on the cleaned and feature-engineered data.

5) **ModelEvaluator**, responsible for evaluating the trained model's performance using appropriate metrics and validation techniques.

Using composition (separate classes coordinated by a higher-level estimator) keeps each step—data loading, cleaning, feature engineering, training, and evaluation—focused on a single responsibility. This improves modularity, testability, and reuse: each component can be developed, swapped, or improved independently without affecting the rest of the pipeline.

## Setup and Imports

Import necessary libraries for data manipulation, visualization, and modeling.

In [1]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from pathlib import Path
from typing import Optional, List

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## 1. DataLoader - Loading TTS Data

We'll start by using our **`DataLoader`** class to load the Tracking the Sun dataset. This class handles the messy details of reading CSV files and filtering them to just the records we care about.

The `DataLoader` class provides:
- Automatic handling of multiple CSV files in a directory
- Date parsing for installation dates
- Year-based filtering (we want 2019-2023)
- Customer segment filtering (commercial and non-residential only)

In [2]:
class DataLoader:
    """
    Data loader for LBNL Tracking the Sun dataset.

    This class handles TTS-specific data loading, cleaning, and filtering
    operations independent of the modeling pipeline.

    Parameters
    ----------
    tts_data_directory : str
        Path to the directory containing the raw TTS data files.

    Attributes
    ----------
    raw_data_directory: Path
        Directory containing the raw TTS data files.

    """

    def __init__(self, tts_data_directory: str):
        self.tts_data_directory = Path(tts_data_directory)
        self.df = None

        self.valid_customer_segments = ['COM', 'RES_MF', 'RES_SF', 'RES', 'AGRICULTURAL',
       'OTHER TAX-EXEMPT', 'GOV', 'SCHOOL', 'NON-RES', 'NON-PROFIT']

    def _filter_by_years(self, df, year_min=None, year_max=None):
        """
        Filter data to specific installation years.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to filter.
        year_min : int, optional
            Minimum installation year to include. If None, includes all years.
        year_max : int, optional
            Maximum installation year to include. If None, includes all years.

        Returns
        -------
        pd.DataFrame
            Filtered dataframe.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if df is None:
            raise ValueError("Data not loaded. Call load_raw() first.")

        if year_min is not None:
            df = df[df.installation_date.dt.year >= year_min]
        if year_max is not None:
            df = df[df.installation_date.dt.year <= year_max]

        return df

    def _filter_by_customer_segment(self, df, segments):
        """
        Filter data to specific customer segments.

        Parameters
        ----------
        segments : list of str
            Customer segments to include (e.g., ['COM', 'NON-RES']).

        Returns
        -------
        pd.DataFrame
            Filtered dataframe.
        segments : list of str
            Customer segments to filter to. If None, includes all segments.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if df is None:
            raise ValueError("Data not loaded. Call load_raw() first.")

        df = df[df['customer_segment'].isin(segments)]

        return df
    
    def _validate_filters(self, year_min, year_max, customer_segments):
        """
        Validate filter parameters.

        Parameters
        ----------
        year_min : int, optional
            Minimum installation year to include. If None, includes all years.
        year_max : int, optional
            Maximum installation year to include. If None, includes all years.
        customer_segments : list of str, optional
            Customer segments to include (e.g., ['COM', 'NON-RES']).

        Raises
        ------
        ValueError
            If year_min is greater than year_max or if customer_segments is not a list of strings.
        """
        if year_min is not None and year_max is not None and year_min > year_max:
            raise ValueError("year_min cannot be greater than year_max.")
        
        if customer_segments is not None:
            if not isinstance(customer_segments, list) or not all(isinstance(seg, str) for seg in customer_segments):
                raise ValueError("customer_segments must be a list of strings.")
            if not set(customer_segments).issubset(set(self.valid_customer_segments)):
                raise ValueError(f"customer_segments must be a subset of {self.valid_customer_segments}.")
            

    def load(
        self,
        year_min: Optional[int] = None,
        year_max: Optional[int] = None,
        customer_segments: Optional[List[str]] = None
    ):
        """
        Load and filter TTS data with common preprocessing steps.

        Parameters
        ----------
        year_min : int, optional
            Minimum year to filter to. If None, includes all years.
        year_max : int, optional
            Maximum year to filter to. If None, includes all years.
        customer_segments : list of str, optional
            Customer segments to filter to. If None, includes all segments.

        Returns
        -------
        pd.DataFrame
            Filtered and cleaned dataframe.
        """

        csvs = list(self.tts_data_directory.glob('*.csv'))

        self._validate_filters(year_min, year_max, customer_segments)

        if csvs:
            self.df = pd.DataFrame()
            for csv in csvs:
                csv_df = pd.read_csv(csv, parse_dates=['installation_date'])

                if year_min is not None or year_max is not None:
                    csv_df = self._filter_by_years(csv_df, year_min, year_max)

                if customer_segments is not None:
                    csv_df = self._filter_by_customer_segment(csv_df, customer_segments)

                self.df = pd.concat([self.df, csv_df], ignore_index=True)
                print(f"Loaded {len(csv_df)} rows from {csv.name}")
        
        else:
            raise ValueError(f"No CSV files found in directory {self.tts_data_directory}")


    def get_data(self):
        """
        Get the current dataframe.

        Returns
        -------
        pd.DataFrame
            Current dataframe.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_raw() or load() first.")

        return self.df

### Instantiate and Load Data

Now we'll create a `DataLoader` instance pointing to our raw data directory and use it to load commercial solar installations from 2019-2023.

In [3]:
tts_dataloader = DataLoader(tts_data_directory='../data/raw')

tts_dataloader.load(year_min=2019, year_max=2023, customer_segments=['COM'])

tts_dataloader.get_data().head()

/var/folders/zj/gmwqw_hj5bj1f90myf__wr_c0000gn/T/ipykernel_27716/3553158664.py:147: DtypeWarning: Columns (1,2,3,11,15,16,18,28,29,31,32,34,35,38,39,40,53,54,56,57,59,60,74,75,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv, parse_dates=['installation_date'])


Loaded 15547 rows from TTS_LBNL_public_file_29-Sep-2025_all.csv


,data_provider_1,data_provider_2,system_ID_1,system_ID_2,installation_date,PV_system_size_DC,total_installed_price,rebate_or_grant,customer_segment,expansion_system,multiple_phase_system,TTS_link_ID,new_construction,tracking,ground_mounted,zip_code,city,state,utility_service_territory,third_party_owned,installer_name,self_installed,azimuth_1,azimuth_2,azimuth_3,tilt_1,tilt_2,tilt_3,module_manufacturer_1,module_model_1,module_quantity_1,module_manufacturer_2,module_model_2,module_quantity_2,module_manufacturer_3,module_model_3,module_quantity_3,additional_modules,technology_module_1,technology_module_2,technology_module_3,BIPV_module_1,BIPV_module_2,BIPV_module_3,bifacial_module_1,bifacial_module_2,bifacial_module_3,nameplate_capacity_module_1,nameplate_capacity_module_2,nameplate_capacity_module_3,efficiency_module_1,efficiency_module_2,efficiency_module_3,inverter_manufacturer_1,inverter_model_1,inverter_quantity_1,inverter_manufacturer_2,inverter_model_2,inverter_quantity_2,inverter_manufacturer_3,inverter_model_3,inverter_quantity_3,additional_inverters,micro_inverter_1,micro_inverter_2,micro_inverter_3,built_in_meter_inverter_1,built_in_meter_inverter_2,built_in_meter_inverter_3,output_capacity_inverter_1,output_capacity_inverter_2,output_capacity_inverter_3,DC_optimizer,inverter_loading_ratio,battery_manufacturer,battery_model,battery_rated_capacity_kW,battery_rated_capacity_kWh,battery_price,technology_type,extensions_multiphase_id
0,Frontier Associates,Texas Central Company,19TCC-001,-1,2019-05-09,29.70,54950.00,3920.0,COM,False,False,-1,0.0,0.0,0.0,78045,Laredo,TX,Texas Central Company,0.0,Peg Solar,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,Sonali Energees USA LLC,SS-330,90.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,Multi-c-Si,-1,-1,0,-1,-1,0,-1,-1,330.0,-1.0,-1.0,0.171875,-1.0,-1.0,-1,-1,2.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1.0,-1.0,-1.0,pv-only,-1
1,Frontier Associates,Texas Central Company,19TCC-004,-1,2019-05-22,24.48,74674.00,19584.0,COM,False,False,-1,0.0,0.0,0.0,78041,Laredo,TX,Texas Central Company,0.0,Peg Solar,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,"Trina Solar Co.,Ltd",-1,72.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1,2.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1.0,-1.0,-1.0,pv-only,-1
2,Frontier Associates,Texas Central Company,19TCC-010,-1,2019-10-08,52.56,181332.00,24390.0,COM,False,False,-1,0.0,0.0,0.0,78570,Mercedes,TX,Texas Central Company,0.0,Ecolectrics,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,SET-Solar,-1,144.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1,1.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1.0,-1.0,-1.0,pv-only,-1
3,Frontier Associates,Texas Central Company,19TCC-018,-1,2019-03-06,23.80,45764.00,19040.0,COM,False,False,-1,0.0,0.0,0.0,78596,Weslaco,TX,Texas Central Company,0.0,Alba Energy,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,"Trina Solar Co.,Ltd",-1,70.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1,2.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1.0,-1.0,-1.0,pv-only,-1
4,Frontier Associates,Texas Central Company,19TCC-068,-1,2019-11-27,307.31,952755.47,46155.5,COM,False,False,-1,0.0,-1.0,1.0,78041,Laredo,TX,Texas Central Company,0.0,Freedom Solar Power Tx,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,SunPower,-1,778.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1,-1,5.0,-1,-1,-1.0,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1.0,-1.0,-1.0,pv-only,-1


### What We Loaded

We successfully loaded 15.5k rows of commercial solar installation data from 2019-2023. Each row represents one solar installation project with details about system size, location, components, and cost.

-----------------------------------

# 📊 EDA: Datatypes and Missing Values

We want to understand the structure of our data before we can clean it and train a model. This includes looking at datatypes, missing values, and distributions of key columns. A bar chart of datatypes will show us which columns are numeric vs categorical, and a barchart display of missing values will reveal which columns have a lot of missing data.

Note: We do have to get ahead of ourselves and do sentinel value replacement before we can do a full EDA, since the dataset uses various sentinel values to indicate missing or invalid data. The next section will handle this cleaning step.

In [4]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=tts_dataloader.get_data().columns,
        y=tts_dataloader.get_data().replace([-1, "-1"], np.nan).isna().mean(),
        marker_color=tts_dataloader.get_data().dtypes.map(lambda dt: 'lightblue' if dt in ['object', 'str'] else 'salmon')
    )
)

fig.update_layout(
    title='Proportion of Missing Values by Column',
    xaxis_title='Column',
    yaxis_title='Proportion of Missing Values',
    xaxis_tickangle=-90,
)

fig.show()

One clear pattern is that for much of the equipment-related columns (e.g. inverter details, battery details), there are a lot of missing values in the secondary and tertiary columns (e.g. inverter_2, inverter_3, battery_2, battery_3) and detail is only provided for the primary equipment (inverter_1, battery_1). This makes sense since most installations likely only have one inverter and one battery, but it also means we will need to drop the secondary and tertiary columns due to the high proportion of missing values.

## 2. DataCleaner - Cleaning the Dataset

Now that we have our raw data loaded, we need to clean it. The **`DataCleaner`** class handles several important cleaning tasks:

- **Sentinel value replacement**: Converts placeholder values like -1 to true NaN
- **Target cleaning**: Removes rows with missing or unrealistically low prices
- **Datatype coercion**: Ensures each column has the appropriate data type
- **Missing data handling**: Drops columns with too many missing values, imputes others
- **Cardinality reduction**: Removes extremely high-cardinality categorical columns that would create too many features

In [82]:
class DataCleaner:
    """
    Data cleaner for LBNL Tracking the Sun dataset.

    This class handles TTS-specific data cleaning operations independent of the modeling pipeline.


    """

    def __init__(self, config_min_target_value=10, config_high_cardinality_threshold=0.05, config_na_drop_thresholds={'string_columns': 0.20, 'numeric_columns': 0.50}):
        """
        Initialize the DataCleaner with configuration parameters.

        Parameters
        ----------
        config_min_target_value : float, optional
            Minimum valid target value. Rows with target values below this will be removed. Default is 10.
        config_high_cardinality_threshold : float, optional
            Proportion of unique values above which a column will be dropped. Default is 0.05 (5%).
        """

        self.df = None

        self.config_min_target_value = config_min_target_value
        self.config_high_cardinality_threshold = config_high_cardinality_threshold
        self.config_na_drop_thresholds = config_na_drop_thresholds

    def load_data(self, df):
        """
        Load data into the cleaner.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.

        Returns
        -------
        None
        """
        self.df = df

    def _make_true_na(self, df):
        """
        Convert common placeholder values for missing data to true NaN.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.

        Returns
        -------
        pd.DataFrame
            Cleaned dataframe with true NaN values.
        """
        df = df.replace([-1, "-1"], np.nan)
        
        return df
    
    def _coerce_datatypes(self, df):
        """
        Coerce datatypes of columns to appropriate types.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.

        Returns
        -------
        pd.DataFrame
            Cleaned dataframe with coerced datatypes.
        """
        for col in df.columns:
            if 'date' in col.lower():
                df[col] = pd.to_datetime(df[col], errors='coerce')

            elif "zip" in col.lower() or "postal" in col.lower():
                df[col] = df[col].astype(str).str.zfill(5)
            elif df[col].dtype in ['bool']:
                df[col] = df[col].astype(int)
            elif df[col].dtype in ['object', 'str']:
                try:
                    df[col] = pd.to_numeric(df[col])
                except Exception:
                    df[col] = df[col].astype('str')
        return df

    def _clean_by_target(self, df, target_col):
        """
        Clean the target variable by removing rows with missing or invalid values.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.
        target_col : str
            Name of the target column to clean.

        Returns
        -------
        pd.DataFrame
            Cleaned dataframe.
        """
        before_count = len(df)
        df = df.dropna(subset=[target_col])
        df = df[df[target_col] >= self.config_min_target_value]
        after_count = len(df)
        print(f"> Removed {before_count - after_count} rows with missing or invalid target values.")
        return df
    
    def _drop_high_na_columns(self, df):
        """
        Drop columns that have a majority of missing values based on configured thresholds.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.

        Returns
        -------
        pd.DataFrame
            Cleaned dataframe with high-NA columns dropped.
        """

        if len(df) == 0:
            print("Warning: Dataframe is empty. Skipping high-NA column drop.")
            return df
        
        string_cols = df.select_dtypes(include=['object']).columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns

        string_na_proportions = df[string_cols].isna().mean()
        numeric_na_proportions = df[numeric_cols].isna().mean()

        cols_to_drop_string = string_na_proportions[string_na_proportions > self.config_na_drop_thresholds['string_columns']].index
        cols_to_drop_numeric = numeric_na_proportions[numeric_na_proportions > self.config_na_drop_thresholds['numeric_columns']].index

        cols_to_drop = list(cols_to_drop_string) + list(cols_to_drop_numeric)
        print(f"> Dropping columns with majority NA values: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)

        return df


    
    def _drop_high_cardinality_columns(self, df):
        """
        Drop columns that have a high proportion of unique values.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.
            
        Returns
        -------
        pd.DataFrame 
            Cleaned dataframe with high-cardinality columns dropped.
        """
        if len(df) == 0:
            print("Warning: Dataframe is empty. Skipping high-cardinality column drop.")
            return df
        
        unique_proportions = df.nunique() / len(df)

        cols_to_drop = unique_proportions[unique_proportions > self.config_high_cardinality_threshold].index
        cols_to_drop = [col for col in cols_to_drop if df[col].dtype in ['object', 'str']]
        print(f"> Dropping high-cardinality columns: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)

        return df
    
    def _drop_single_value_columns(self, df):
        """
        Drop columns that have only a single unique value.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to clean.
            
        Returns
        -------
        pd.DataFrame 
            Cleaned dataframe with single-value columns dropped.
        """
        single_value_cols = df.columns[df.nunique() <= 1]
        print(f"> Dropping single-value columns: {list(single_value_cols)}")
        df = df.drop(columns=single_value_cols)

        return df
    
    def clean(self, target_col='total_installed_price'):
        """
        Perform all cleaning steps on the loaded dataframe.

        Parameters
        ----------
        target_col : str, optional
            Target column to clean. Default is 'total_installed_price'.
        min_target_value : float, optional
            Minimum valid target value. Default is 10.

        Returns
        -------
        pd.DataFrame
            Cleaned dataframe.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_data() first.")

        self.df = self._make_true_na(self.df)
        self.df = self._clean_by_target(self.df, target_col)
        self.df = self._drop_high_na_columns(self.df)
        self.df = self._drop_single_value_columns(self.df)
        self.df = self._drop_high_cardinality_columns(self.df)
        self.df = self._coerce_datatypes(self.df)
        return self.df

### Apply Data Cleaning

Let's instantiate our `DataCleaner`, load our data into it, and apply all cleaning operations with a single `.clean()` call.


In [85]:
data_cleaner_config = {
    'config_min_target_value': 10,
    'config_high_cardinality_threshold': 0.10,
    'config_na_drop_thresholds': {'string_columns': 0.10, 'numeric_columns': 0.50}
}

tts_cleaner = DataCleaner(**data_cleaner_config)
tts_cleaner.load_data(tts_dataloader.get_data())
cleaned_df = tts_cleaner.clean(target_col='total_installed_price')

cleaned_df.shape

> Removed 5142 rows with missing or invalid target values.
> Dropping columns with majority NA values: ['data_provider_2', 'system_ID_2', 'TTS_link_ID', 'module_model_1', 'module_manufacturer_2', 'module_model_2', 'module_manufacturer_3', 'module_model_3', 'technology_module_1', 'technology_module_2', 'technology_module_3', 'inverter_manufacturer_1', 'inverter_model_1', 'inverter_manufacturer_2', 'inverter_model_2', 'inverter_manufacturer_3', 'inverter_model_3', 'battery_manufacturer', 'battery_model', 'extensions_multiphase_id', 'new_construction', 'azimuth_2', 'azimuth_3', 'tilt_2', 'tilt_3', 'module_quantity_2', 'module_quantity_3', 'BIPV_module_2', 'BIPV_module_3', 'bifacial_module_2', 'bifacial_module_3', 'nameplate_capacity_module_2', 'nameplate_capacity_module_3', 'efficiency_module_2', 'efficiency_module_3', 'inverter_quantity_2', 'inverter_quantity_3', 'micro_inverter_2', 'micro_inverter_3', 'built_in_meter_inverter_2', 'built_in_meter_inverter_3', 'output_capacity_inverter_2'

(10405, 30)

### What We Cleaned

The `DataCleaner` successfully processed our dataset and dropped the high-cardinality `zip_code` column. Our data is now clean, properly typed, and ready for feature engineering.

-----------------------

## 3. FeatureEngineer - Creating Useful Features

With clean data in hand, we'll use our **`FeatureEngineer`** class to create new features that help our model learn better patterns.

Currently, our feature engineer creates:
- **`days_since_2000`**: A temporal feature representing how many days after January 1, 2000 the system was installed. This captures time trends in solar pricing more effectively than raw dates.

In [86]:
class FeatureEngineer:
    """
    Feature engineer for LBNL Tracking the Sun dataset.

    This class handles TTS-specific feature engineering operations independent of the modeling pipeline.

    Parameters
    ----------
    None

    Attributes
    ----------
    None

    """

    def __init__(self):
        pass

    def add_day_count(self, df):
        """
        Add a feature for the number of days since installation.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to engineer.

        Returns
        -------
        pd.DataFrame
            Dataframe with new 'days_since_2000' feature.
        """
        df['days_since_2000'] = (df['installation_date'] - pd.Timestamp('2000-01-01')).dt.days
        
        return df
    
    def combine_module_counts(self, df):
        """
        Combine module count features into a single feature.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to engineer.

        Returns
        -------
        pd.DataFrame
            Dataframe with new 'total_module_count' feature.
        """
        module_cols = [col for col in df.columns if 'module_quantity' in col.lower()]
        df['total_module_count'] = df[module_cols].replace(np.nan, 0).sum(axis=1)

        df = df.drop(columns=module_cols)
        
        return df

### Apply Feature Engineer

Below, we instantiate our `FeatureEngineer` and apply it to our cleaned dataset to create the new features. This generates two new features:
* `days_since_2000`: A numeric feature representing how many days after January 1, 2000 the system was installed. This captures time trends in solar pricing more effectively than raw dates.
* `total_module_count`: A numeric feature representing the total number of modules in the system.

In [87]:
feature_engineer = FeatureEngineer()

engineered_df = feature_engineer.add_day_count(cleaned_df)
engineered_df = feature_engineer.combine_module_counts(engineered_df)

---------------------

## 4. Preprocessor - Preparing Features for Modeling

Now we need to transform our features into a format that our machine learning model can understand. The **`Preprocessor`** class automatically:

1. **Sorts columns** by type (numerical, binary, low-cardinality categorical, high-cardinality categorical)
2. **Builds a sklearn ColumnTransformer** that applies the right transformation to each column type:
   - **One-hot encoding** for low-cardinality categoricals (creates binary columns)
   - **Target encoding** for high-cardinality categoricals (replaces categories with mean target value)
   - **Passthrough** for binary columns (already in good format)
   - **Standard scaling** for numerical features (normalizes to mean=0, std=1)

Taken together, this preprocessing pipeline ensures that all our features are in the right format for modeling, while also reducing dimensionality and improving model performance.



In [105]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

class Preprocessor():
    """
    Preprocessor for our Capex solar cost estimation model.
    """

    def __init__(self, target_col='total_installed_price'):
        """Initialize the Preprocessor with configuration parameters."""
        self.target_col = target_col
        self.columns = None

        self.preprocessor = None

    def _sort_columns(self, df: pd.DataFrame):
        """Sort columns into numerical, binary, low-cardinality categorical, and high-cardinality categorical based on datatypes and unique value counts.
        
        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to use for determining column types.
        """
        self.columns = dict(
            target_col = self.target_col,
            num_cols = [],
            binary_cols = [],
            cat_low_card_cols = [],
            cat_high_card_cols = []
        )
        
        for col in df.columns:
            if col == self.columns['target_col']:
                continue
            elif df[col].dtype in ['int64', 'float64']:
                self.columns['num_cols'].append(col)
            elif df[col].dtype == 'bool' or (df[col].nunique() == 2):
                self.columns['binary_cols'].append(col)
            elif df[col].dtype in ['str', 'object', 'category']:
                if df[col].nunique() < 10:
                    self.columns['cat_low_card_cols'].append(col)
                else:
                    self.columns['cat_high_card_cols'].append(col)

    def get_feature_names(self):
        """Get feature names after preprocessing"""
        if self.preprocessor is None:
            raise ValueError("Preprocessor not built. Must be fit to data first.")
        return self.preprocessor.get_feature_names_out()

    def build_preprocessor(self, df):
        """
        Build the preprocessing pipeline based on the dataframe's columns and datatypes.
        
        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to use for determining column types and building the preprocessor.
        
        Returns
        -------
        ColumnTransformer
            The built preprocessing pipeline.
        """
        self._sort_columns(df)
        
        low_card_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]).set_output(transform="pandas")
        
        high_card_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', TargetEncoder())
        ]).set_output(transform="pandas")
        
        binary_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
]).set_output(transform="pandas")
        
        num_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]).set_output(transform="pandas")
        
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('cat_low_card', low_card_pipeline, self.columns['cat_low_card_cols']),
                ('cat_high_card', high_card_pipeline, self.columns['cat_high_card_cols']),
                ('binary', binary_pipeline, self.columns['binary_cols']),
                ('num', num_pipeline, self.columns['num_cols']),
            ], 
            remainder='drop'
        ).set_output(transform="pandas")
        
        return self.preprocessor

In [106]:
preprocessor = Preprocessor(target_col='total_installed_price')
preprocessor.build_preprocessor(engineered_df)
preprocessor.preprocessor.fit(engineered_df.drop(columns=['total_installed_price']), engineered_df[preprocessor.target_col])

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat_low_card', ...), ('cat_high_card', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and 

# 📊 EDA: Identifying Relevant Features

We can use scikit-learn's `SelectKBest` feature selection method to identify which features are most strongly correlated with our target variable (total installed price). This will help us understand which features are most important for predicting solar CAPEX and may also allow us to reduce the number of features we use in our model for better performance and interpretability.

In [104]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=10)

preprocessed_df = preprocessor.preprocessor.fit_transform(engineered_df.drop(columns=['total_installed_price']), engineered_df[preprocessor.target_col])

selector.fit(preprocessed_df, engineered_df[preprocessor.target_col])

scores = selector.scores_
feature_names = preprocessor.preprocessor.get_feature_names_out()

feature_scores = pd.DataFrame({'feature': feature_names, 'score': scores})
feature_scores = feature_scores.sort_values(by='score', ascending=False)

In [95]:
fig = go.Figure(data=[go.Bar(x=feature_scores['feature'], y=feature_scores['score'])])
fig.update_layout(title='Feature Scores from SelectKBest', xaxis_title='Feature', yaxis_title='Score', xaxis_tickangle=-45)
fig.show()

----------------

## Model Pipeline


AttributeError: 'NoneType' object has no attribute 'fit'